In [1]:
import pandas as pd
import torch
import os

from torch.utils.data import Dataset
from transformers import (BertTokenizer, BertForSequenceClassification, Trainer,
                          TrainingArguments, BertPreTrainedModel)
# from simpletransformers.language_modeling import LanguageModelingModel
from sklearn.metrics import accuracy_score, f1_score


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))

True
NVIDIA GeForce RTX 3060
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12050MB, multi_processor_count=28)


In [3]:
MODEL_NAME = '/home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert'
TRAIN_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/SAIL_data/BN_data_train.tsv'
TEST_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/SAIL_data/BN_data_test.tsv'
EVAL_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/SAIL_data/BN_data_dev.tsv'


In [4]:
def tsv_to_text(tsv_file_loc):
    file_name = tsv_file_loc.split("/")[-1].split(".")[0]
    txt_name = tsv_file_loc.replace(".tsv", ".txt")
    txt_name = txt_name.replace("split_merged", "texts")

    if os.path.exists(txt_name):
        return [txt_name, file_name]

    df = pd.read_csv(tsv_file_loc, sep="\t")

    for txt in df["text"]:
        with open(txt_name, "a", encoding="utf8") as f:
            f.writelines(txt + "\n")
    return [txt_name, file_name]


tsv_to_text(TRAIN_FILE_LOC)


['/home/abdullah/Code/dl/bnlp-resources/sentiment/SAIL_data/BN_data_train.txt',
 'BN_data_train']

In [5]:
def tsv_to_df(csv_file_loc):
    df = pd.read_csv(csv_file_loc, sep='\t')
    # remove id column
    df = df.drop(columns=['id'])

    # replace neutral with 0, positive with 1, negative with 2 in class_label column
    df.loc[df['class'] == 'BN_NEU', 'class'] = 0
    df.loc[df['class'] == 'BN_POS', 'class'] = 1
    df.loc[df['class'] == 'BN_NEG', 'class'] = 2

    return df


train_df = tsv_to_df(TEST_FILE_LOC)

texts = train_df['text'].tolist()
print(train_df)


                                                  text class
0    'পকেটে শখানেক টাকা ... আর এই মুহুর্তে দিনব্যাপ...     2
1    'শেষ পর্যায়ে এসে গল্পটা এভাবে ভুল পথে মোড় না...     2
2    'সংসদেও ঘুমালেন সমাজকল্যাণ মন্ত্রী  http://t.c...     2
3    'সীমান্তে নারী ও শিশুসহ আটক ৮ http://t.co/Fz3d...     2
4    'আমিও মানুষ ভালা না, মনে মনে শুয়োরের বাচ্চা বল...     2
..                                                 ...   ...
199  'আবের সঙ্গে সৌজন্য সাক্ষাৎ করেছেন খালেদা জিয়া...     1
200  'ভৈরব নদ সংস্কার ও খননের দাবিতে জনউদ্যোগের মান...     1
201  'আলো টেলিফিল্মটা দেখলাম। মাঝেমধ্যে পর্যাপ্ত পর...     1
202      '@arif_rony2 তারপরে ও দেখতে পেলে খুব মজা পাই'     1
203  'চল কাবার পানে ওহে মুহাজির,পেতে প্রেম শুধা আল্...     1

[204 rows x 2 columns]


In [6]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }


In [7]:
def f1_calculator(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  f1 = f1_score(labels, preds, average='weighted')
  return {
      'f1': f1,
  }


In [8]:
train_list = tsv_to_text(TRAIN_FILE_LOC)
test_list = tsv_to_text(TEST_FILE_LOC)
eval_list = tsv_to_text(EVAL_FILE_LOC)


In [9]:
os.environ["WANDB_DISABLED"] = "true"

train_df = tsv_to_df(TRAIN_FILE_LOC)
test_df = tsv_to_df(TEST_FILE_LOC)
eval_df = tsv_to_df(EVAL_FILE_LOC)


class TINYDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(value[idx])
                for key, value in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_df['text'].tolist(
), padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_df['text'].tolist(
), padding=True, truncation=True, max_length=512)
eval_encodings = tokenizer(eval_df['text'].tolist(
), padding=True, truncation=True, max_length=512)

train_dataset = TINYDataset(train_encodings, train_df['class'].tolist())
test_dataset = TINYDataset(test_encodings, test_df['class'].tolist())
eval_dataset = TINYDataset(eval_encodings, eval_df['class'].tolist())

model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to('cuda')
model.manual_seed = 14


Some weights of the model checkpoint at /home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.1.intermediate.dense.weight_mask', 'bert.encoder.layer.1.attention.output.dense.weight_mask', 'bert.encoder.layer.0.intermediate.dense.weight_orig', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.encoder.layer.1.attention.self.query.weight_mask', 'bert.encoder.layer.1.intermediate.dense.weight_orig', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'bert.encoder.layer.0.attention.output.dense.weight_orig', 'bert.encoder.layer.0.attention.output.dense.weight_mask', 'bert.encoder.layer.1.output.dense.weight_orig', 'bert.encoder.layer.0.intermediate.dense.weight_mask', 'bert.encoder.layer.0.attention.self.value.weight_mask', 'bert.encoder.layer.1.attention.self.value.weight_orig', 'bert.encoder.layer.0.attention.self.key.weight_mask', 'bert.encoder.lay

## 1 Epoch Train

In [10]:
training_args = TrainingArguments(
    output_dir=f"temp",
    num_train_epochs=1,
    per_device_train_batch_size=48,
    per_device_eval_batch_size=96,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    overwrite_output_dir=True,
    logging_dir=f"temp/logs",
    logging_steps=15,
    save_steps=15,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    seed=14,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset

)

trainer.train()


Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
***** Running training *****
  Num examples = 697
  Num Epochs = 1
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 1
  Total optimization steps = 15
100%|██████████| 15/15 [00:00<00:00, 33.87it/s]***** Running Evaluation *****
  Num examples = 98
  Batch size = 96


{'loss': 1.099, 'learning_rate': 1.5e-06, 'epoch': 1.0}


                                               
100%|██████████| 15/15 [00:00<00:00, 33.87it/s]Saving model checkpoint to temp/checkpoint-15
Configuration saved in temp/checkpoint-15/config.json


{'eval_loss': 1.0983589887619019, 'eval_runtime': 0.0171, 'eval_samples_per_second': 5743.214, 'eval_steps_per_second': 117.208, 'epoch': 1.0}


Model weights saved in temp/checkpoint-15/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-15 (score: 1.0983589887619019).
100%|██████████| 15/15 [00:01<00:00,  8.84it/s]

{'train_runtime': 1.7031, 'train_samples_per_second': 409.253, 'train_steps_per_second': 8.807, 'train_loss': 1.0989625295003256, 'epoch': 1.0}


TrainOutput(global_step=15, training_loss=1.0989625295003256, metrics={'train_runtime': 1.7031, 'train_samples_per_second': 409.253, 'train_steps_per_second': 8.807, 'train_loss': 1.0989625295003256, 'epoch': 1.0})

## 100 Epoch with 1 epoch model

In [12]:
MODEL_NAME = '/home/abdullah/Code/dl/lt_bert/best_models/1/1_sail_sentiment_50'
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=3).to('cuda')
model.manual_seed = 14

for name, param in model.named_parameters():
    if 'bert.encoder.layer.0' in name:
        # print(name, param.requires_grad)
        param.requires_grad = False
# print("\n\n\n")
# for name, param in model.named_parameters():
#     print(name, param.requires_grad)

training_args = TrainingArguments(
    output_dir=f"temp",
    num_train_epochs=100,
    per_device_train_batch_size=48,
    per_device_eval_batch_size=96,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    overwrite_output_dir=True,
    logging_dir=f"temp/logs",
    logging_steps=100,
    save_steps=100,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    seed=14,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=f1_calculator,

)

trainer.train()

# print(cool.metrics)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()


loading configuration file /home/abdullah/Code/dl/lt_bert/best_models/1/1_sail_sentiment_50/config.json
Model config BertConfig {
  "_name_or_path": "/home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "voc

{'loss': 1.0976, 'learning_rate': 1e-05, 'epoch': 6.67}


                                                  
  7%|▋         | 100/1500 [00:01<00:19, 71.66it/s]Saving model checkpoint to temp/checkpoint-100
Configuration saved in temp/checkpoint-100/config.json


{'eval_loss': 1.0944733619689941, 'eval_f1': 0.1973804447151995, 'eval_runtime': 0.0154, 'eval_samples_per_second': 6367.213, 'eval_steps_per_second': 129.943, 'epoch': 6.67}


Model weights saved in temp/checkpoint-100/pytorch_model.bin
 13%|█▎        | 200/1500 [00:03<00:17, 75.65it/s]***** Running Evaluation *****
  Num examples = 98
  Batch size = 96


...


100%|██████████| 1500/1500 [00:30<00:00, 75.02it/s]Saving model checkpoint to temp/checkpoint-1500
Configuration saved in temp/checkpoint-1500/config.json


{'eval_loss': 1.6162593364715576, 'eval_accuracy': 0.47959183673469385, 'eval_runtime': 0.0139, 'eval_samples_per_second': 7033.208, 'eval_steps_per_second': 143.535, 'epoch': 100.0}


Model weights saved in temp/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-300 (score: 1.0742554664611816).
100%|██████████| 1500/1500 [00:31<00:00, 47.55it/s]

{'train_runtime': 31.5273, 'train_samples_per_second': 2210.785, 'train_steps_per_second': 47.578, 'train_loss': 0.7398870595296224, 'epoch': 100.0}


TrainOutput(global_step=1500, training_loss=0.7398870595296224, metrics={'train_runtime': 31.5273, 'train_samples_per_second': 2210.785, 'train_steps_per_second': 47.578, 'train_loss': 0.7398870595296224, 'epoch': 100.0})